In [138]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import os


In [204]:
#import the meta data with tmdb_id and imdb_id
df_meta = pd.read_csv('data/movies_metadata.csv', usecols = ['id', 'imdb_id','genres','title'])
df_meta.rename(columns={'id': 'tmdb_id'}, inplace=True)

#clean TBDB numbers, convert to ints, remove non-integres. 3 NaNs converted to zero
for index, row in df_meta.iterrows():    
        try:
            row['tmdb_id'] = int(row['tmdb_id'])
        except ValueError:
            row['tmdb_id'] = 0

#clean IMBD numbers, convert to ints, 20 NaNs converted to zero
for index, row in df_meta.iterrows():
        if isinstance(row['imdb_id'], str) == True and row['imdb_id'][0:2] == "tt":
            row['imdb_id'] = int(row['imdb_id'][2:])
        else:
            row['imdb_id'] = 0
            
for index, row in df_meta.iterrows():
    if row['imdb_id'] == 0 and row['tmdb_id'] == 0:
        df_meta.drop(index, inplace=True)

df_meta

,genres,tmdb_id,imdb_id,title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,114709,Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,113497,Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,113228,Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,114885,Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",11862,113041,Father of the Bride Part II
...,...,...,...,...
45461,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,6209470,Subdue
45462,"[{'id': 18, 'name': 'Drama'}]",111109,2028550,Century of Birthing
45463,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,303758,Betrayal
45464,[],227506,8536,Satan Triumphant


In [205]:
print(df_meta['tmdb_id'].nunique())
print(df_meta['imdb_id'].nunique())

45433
45417


In [203]:
# clean the links file

df_links = pd.read_csv('data/links.csv')

df_links.rename(columns={'imdbId': 'imdb_id', 'tmdbId': 'tmdb_id'}, inplace=True)

df_links['movieId'] = df_links['movieId'].astype(int) #convert strings to int
df_links['imdb_id'] = df_links['imdb_id'].astype(int) #convert strings to int
df_links['tmdb_id'].fillna(0, inplace=True)

# clean TMDB, convert to ints, NaNs
for index, row in df_links.iterrows():    
        try:
            row['tmdb_id'] = int(row['tmdb_id']) 
        except ValueError:
            row['tmdb_id'] = 0

df_links['tmdb_id'] = df_links['tmdb_id'].astype(int) #convert strings to int

for index, row in df_links.iterrows():
    if row['imdb_id'] == 0 and row['tmdb_id'] == 0:
        df_links.drop(index, inplace=True)
        print("dropped " + str(index))

df_links

,movieId,imdb_id,tmdb_id
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862
...,...,...,...
45838,176269,6209470,439050
45839,176271,2028550,111109
45840,176273,303758,67758
45841,176275,8536,227506


In [206]:
# iterate throgh the links csv and lookup the imdb numbers in the meta csv.
# if the imdfb mathces then pull the title and genres form file
# if the imdb number does not exist then search the tmdb 

df_links['title'] = ""
df_links['genre'] = ""

for index, row in df_links.iterrows():
    if index%100 == 0:
        print(index)
    
    i_result = df_meta.loc[df_meta['imdb_id'] == row['imdb_id']]
    t_result = df_meta.loc[df_meta['tmdb_id'] == row['tmdb_id']]
    
    if len(i_result) != 0:
        df_links.at[index,'title'] = i_result.iloc[0,3]
        df_links.at[index,'genre'] = i_result.iloc[0,0]
    
    elif len(t_result) != 0:
        df_links.at[index,'title'] = t_result.iloc[0,3]
        df_links.at[index,'genre'] = t_result.iloc[0,0]
    
    else:
        df_links.at[index,'title'] = ""
        df_links.at[index,'genre'] = ""


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [207]:
df_links

,movieId,imdb_id,tmdb_id,title,genre
0,1,114709,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,2,113497,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,3,113228,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,4,114885,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,5,113041,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...,...
45838,176269,6209470,439050,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n..."
45839,176271,2028550,111109,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]"
45840,176273,303758,67758,Betrayal,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45841,176275,8536,227506,Satan Triumphant,[]


In [212]:
df_links.to_csv('meta_clean.csv', encoding='utf-8', index=False)